In [1]:
include("/Users/alamf/1Jupyter/current_projects/DQC.jl/src/models.jl")
include("/Users/alamf/1Jupyter/current_projects/DQC.jl/src/train_sqc.jl")
include("/Users/alamf/1Jupyter/current_projects/DQC.jl/src/train_dqc.jl")

dqc_svd

In [2]:
n = 2
circ = rn_layer(rand(n,3), 1:n)
hilbert = siteinds("Qubit", n)
mps = MPS(hilbert, [1,1])

# run works as expected
println(norm(mps_to_vec(run(mps, circ)) - kron(circ[2][3].mat, circ[1][3].mat) * [1,0,0,0]))

# run with cc = true works as expected
println(norm(mps_to_vec(run(mps, circ; cc=true)) - kron(circ[2][3].mat', circ[1][3].mat') * [1,0,0,0]))

8.441528768080324e-17
0.0


In [3]:
n = 4
circ = rn_layer(rand(n,3), 1:n)
hilbert = siteinds("Qubit", n)
mps_a = randomMPS(hilbert;linkdims=2)
mps_b = randomMPS(hilbert;linkdims=2);

# cc = true works as expected 
overlap = inner(run(mps_b, []; cc=true), run(mps_a, circ))
println(overlap - inner(run(mps_b, circ; cc=true), run(mps_a, [])))
println(overlap - inner(run(mps_b, circ[4:4]; cc=true), run(mps_a, circ[1:3])))

# contract_all works as expected 
mps_e, mps_f = run(mps_b, circ[4:4]; cc=true), run(mps_a, circ[1:3])
println(overlap - contract_all(conj.(mps_e), mps_f)[1])

-9.71445146547012e-17 - 1.1102230246251565e-16im
5.551115123125783e-17 + 0.0im
5.551115123125783e-17 + 0.0im


In [4]:
# testing 1 qubit environments

n = 4
circ = rn_layer(rand(n,3), 1:n)
hilbert = siteinds("Qubit", n)
left_mps = randomMPS(hilbert;linkdims=2)
right_mps = randomMPS(hilbert;linkdims=2)
overlap = inner(right_mps, runcircuit(left_mps, circ))

for i in 1:length(circ)
    env = environment(left_mps, right_mps, circ, i)
    println(tr(env * circ[i][3].mat) - overlap)
end

8.326672684688674e-17 - 5.551115123125783e-17im
5.551115123125783e-17 + 0.0im
1.3877787807814457e-16 + 5.551115123125783e-17im
5.551115123125783e-17 + 2.7755575615628914e-17im


In [5]:
# testing 2 qubit environments

n = 4
circ = kak_circuit(rand(2,n-1,15))
hilbert = siteinds("Qubit", n)
left_mps = randomMPS(hilbert;linkdims=2)
right_mps = randomMPS(hilbert;linkdims=2)
overlap = inner(right_mps, runcircuit(left_mps, circ))

for i in 1:length(circ)
    env = environment(left_mps, right_mps, circ, i)
    println(tr(env * circ[i][3].mat) - overlap)
end

-1.214306433183765e-17 - 1.3877787807814457e-16im
-1.457167719820518e-16 - 1.3877787807814457e-16im
-3.469446951953614e-17 - 1.942890293094024e-16im
-2.7755575615628914e-17 - 1.942890293094024e-16im
3.469446951953614e-18 - 1.942890293094024e-16im
-6.245004513516506e-17 - 1.249000902703301e-16im


In [6]:
# testing environments with dressed MPSs
n = 6
circ = rn_layer(rand(n,3), 1:n)
hilbert = siteinds("Qubit", n)
left_mps = randomMPS(hilbert;linkdims=2)

right_mps = randomMPS(siteinds("Qubit", 4);linkdims=2)
for loc in [3,4]
    right_mps = insert_zero_ket(right_mps, loc)
end
right_mps = rename_indices(right_mps, hilbert)
overlap = inner(right_mps, runcircuit(left_mps, circ))

for i in 1:length(circ)
    env = environment(left_mps, right_mps, circ, i)
    println(tr(env * circ[i][3].mat) - overlap)
end

-6.245004513516506e-17 - 9.71445146547012e-17im
-9.020562075079397e-17 - 1.249000902703301e-16im
-7.632783294297951e-17 - 7.632783294297951e-17im
-4.85722573273506e-17 - 4.85722573273506e-17im
6.938893903907228e-18 + 1.3877787807814457e-17im
1.3877787807814457e-17 + 1.3877787807814457e-17im


In [7]:
# testing SVD updates
for i in 1:length(circ)
    env = environment(left_mps, right_mps, circ, i)
    println(abs2(tr(env * circ[i][3].mat)))

    new_circ = svd_update(env, circ, i)
    env = environment(left_mps, right_mps, new_circ, i)
    println(abs2(tr(env * circ[i][3].mat)))
    println("")
end

0.004678043899542321
0.010605707978334268

0.010605707978334267
0.01285102956771538

0.012851029567715397
0.02743170001623792

0.02743170001623795
0.11489896947336468

0.11489896947336464
0.33483002589959115

0.33483002589959143
0.4783336145047447



In [8]:
# testing training of static circuit
n = 8
targ_mps = tfim_gs(n)
n_layers = 1
ansatz = generate_sqc_ansatz(targ_mps, n_layers; data="gates")
filename = "debug.jld"
cost_list, circ = sqc_svd(targ_mps, ansatz, filename; num_sweeps=100, quiet=false);

Progress:  49%|█████████████████████                    |  ETA: 0:00:07

0.06414189831862382

Progress:  51%|█████████████████████                    |  ETA: 0:00:07

Progress: 100%|█████████████████████████████████████████| Time: 0:00:1039m


0.06414189831862394


In [2]:
n = 4
targ_mps = tfim_gs(n)
n_layers = 1
n_clusters = 2
ansatz = generate_dqc_ansatz(targ_mps, n_clusters, n_layers)
(hilbert, meas_qubits, active_qubits, pre_circ, post_circ_list) = ansatz
t_ensemble = targ_ensemble(targ_mps, meas_qubits, hilbert, post_circ_list);

In [3]:
# testing dqc_cost and dqc_environment
c = dqc_cost(pre_circ, hilbert, meas_qubits, t_ensemble) 
for gate_idx in 1:length(pre_circ)
    dqc_env = dqc_environment(pre_circ, hilbert, meas_qubits, gate_idx, t_ensemble)
    c_bound = (4 - abs(tr(dqc_env * pre_circ[gate_idx][3].mat)))/4
    println([c,c_bound])
end

[0.6320373123214988, 0.8295035847306174]
[0.6320373123214988, 0.8295035847306174]
[0.6320373123214988, 0.8295035847306174]
[0.6320373123214988, 0.8295035847306176]
[0.6320373123214988, 0.8295035847306176]


In [4]:
# testing optim_pre_circ 
old_c = dqc_cost(pre_circ, hilbert, meas_qubits, t_ensemble) 
pre_circ = optim_pre_circ(pre_circ, hilbert, meas_qubits, t_ensemble)
new_c = dqc_cost(pre_circ, hilbert, meas_qubits, t_ensemble) 
println((old_c, new_c))

(0.6320373123214988, 0.07504872126371984)


In [5]:
# testing optim_post_circ
mps = runcircuit(hilbert, pre_circ)
proj_mps_list, meas_outcomes_list = all_samples(mps, meas_qubits; new_inds=siteinds(targ_mps))

for i in 1:4
    post_circ_list[i] = optim_post_circ(post_circ_list[i], proj_mps_list[i], targ_mps, active_qubits)
    t_ensemble = targ_ensemble(targ_mps, meas_qubits, hilbert, post_circ_list)
    c = dqc_cost(pre_circ, hilbert, meas_qubits, t_ensemble) 
    println(c)
end

0.06572429298357876
0.0528243203675447
0.04715902052949775
0.034029025687655506


In [3]:
# testing dqc_svd
n = 4
targ_mps = tfim_gs(n)
n_layers = 1
n_clusters = 2
ansatz = generate_dqc_ansatz(targ_mps, n_clusters, n_layers)
filename = "debug.jld"
cost_list, pre_circ, post_circ_list, elapsed_time = dqc_svd(targ_mps, ansatz, filename; num_sweeps=50);

Progress:  20%|█████████                                |  ETA: 0:00:14

0.012424018646322949


Progress:  40%|█████████████████                        |  ETA: 0:00:10

0.011671405475608632


Progress:  60%|█████████████████████████                |  ETA: 0:00:07

0.011334763577059037


Progress:  80%|█████████████████████████████████        |  ETA: 0:00:04

0.011203477360536868


Progress: 100%|█████████████████████████████████████████| Time: 0:00:17


0.011155448534124202
